# 🌐 AI Fashion Assistant v2.0 - Hybrid Space Creation

**Phase 2, Notebook 3/3** - Final notebook of Phase 2

---

## 🎯 Objectives

1. Load text and image embeddings
2. Create hybrid space (text + image concatenation)
3. Build FAISS index for fast retrieval
4. Validate index quality
5. Test search functionality

---

## 📊 Input Files

```
embeddings/
├── text/
│   └── combined_1536d_normalized.npy
└── image/
    └── clip_image_768d_normalized.npy
```

---

## 🎯 Output

```
embeddings/
└── hybrid/
    ├── hybrid_2304d.npy          (44,417 x 2304)
    └── hybrid_2304d_normalized.npy

indexes/
└── faiss_hybrid_hnsw.index       (~500 MB)
```

---

## 📋 Quality Gates

- ✓ Hybrid embeddings: 2304d (1536 + 768)
- ✓ FAISS index built successfully
- ✓ Index size reasonable (~500 MB)
- ✓ Search returns results

---

In [1]:
# ============================================================
# 1) SETUP
# ============================================================

from google.colab import drive
drive.mount("/content/drive", force_remount=False)

# Check GPU (optional for this notebook)
import torch
print(f"GPU available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

Mounted at /content/drive
GPU available: True
GPU: NVIDIA A100-SXM4-40GB


In [2]:
# ============================================================
# 2) INSTALL FAISS
# ============================================================

print("📦 Installing FAISS...\n")

# Install faiss-cpu (faiss-gpu for GPU support)
!pip install -q faiss-cpu
# For GPU: !pip install -q faiss-gpu

print("\n✅ FAISS installed!")

📦 Installing FAISS...

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 107.8 MB/s eta 0:00:00

✅ FAISS installed!


In [3]:
# ============================================================
# 3) IMPORTS
# ============================================================

import numpy as np
import pandas as pd
from pathlib import Path
import json
import time
import faiss
from typing import List, Tuple
from tqdm.auto import tqdm

import warnings
warnings.filterwarnings('ignore')

print("✅ All imports successful!")
print(f"   FAISS version: {faiss.__version__}")

✅ All imports successful!
   FAISS version: 1.13.1


In [4]:
# ============================================================
# 4) PATHS & CONFIG
# ============================================================

PROJECT_ROOT = Path("/content/drive/MyDrive/ai_fashion_assistant_v2")
EMB_DIR = PROJECT_ROOT / "embeddings"
EMB_TEXT_DIR = EMB_DIR / "text"
EMB_IMAGE_DIR = EMB_DIR / "image"
EMB_HYBRID_DIR = EMB_DIR / "hybrid"
INDEX_DIR = PROJECT_ROOT / "indexes"

# Create directories
EMB_HYBRID_DIR.mkdir(parents=True, exist_ok=True)
INDEX_DIR.mkdir(parents=True, exist_ok=True)

print("📁 Directories:")
print(f"   Embeddings: {EMB_DIR}")
print(f"   Hybrid: {EMB_HYBRID_DIR}")
print(f"   Indexes: {INDEX_DIR}")

📁 Directories:
   Embeddings: /content/drive/MyDrive/ai_fashion_assistant_v2/embeddings
   Hybrid: /content/drive/MyDrive/ai_fashion_assistant_v2/embeddings/hybrid
   Indexes: /content/drive/MyDrive/ai_fashion_assistant_v2/indexes


In [5]:
# ============================================================
# 5) LOAD EMBEDDINGS
# ============================================================

print("📂 LOADING EMBEDDINGS...\n")
print("=" * 80)

# Load text embeddings (normalized)
print("Loading text embeddings...")
text_emb_path = EMB_TEXT_DIR / "combined_1280d_normalized.npy"
if not text_emb_path.exists():
    # Try non-normalized
    text_emb_path = EMB_TEXT_DIR / "combined_1536d.npy"
    print(f"  Using non-normalized version")

text_embeddings = np.load(text_emb_path)
print(f"✅ Text embeddings loaded")
print(f"   Path: {text_emb_path.name}")
print(f"   Shape: {text_embeddings.shape}")
print(f"   Size: {text_emb_path.stat().st_size / 1024**2:.1f} MB")

# Load image embeddings (normalized)
print("\nLoading image embeddings...")
image_emb_path = EMB_IMAGE_DIR / "clip_image_768d_normalized.npy"
image_embeddings = np.load(image_emb_path)
print(f"✅ Image embeddings loaded")
print(f"   Path: {image_emb_path.name}")
print(f"   Shape: {image_embeddings.shape}")
print(f"   Size: {image_emb_path.stat().st_size / 1024**2:.1f} MB")

print("\n" + "=" * 80)
print(f"✅ All embeddings loaded!")
print(f"   Total products: {len(text_embeddings):,}")

📂 LOADING EMBEDDINGS...

Loading text embeddings...
✅ Text embeddings loaded
   Path: combined_1280d_normalized.npy
   Shape: (44417, 1536)
   Size: 260.3 MB

Loading image embeddings...
✅ Image embeddings loaded
   Path: clip_image_768d_normalized.npy
   Shape: (44417, 768)
   Size: 130.1 MB

✅ All embeddings loaded!
   Total products: 44,417


In [6]:
# ============================================================
# 6) CREATE HYBRID EMBEDDINGS
# ============================================================

print("🔗 CREATING HYBRID EMBEDDINGS...\n")
print("=" * 80)

# Validate shapes match
assert len(text_embeddings) == len(image_embeddings), \
    f"Shape mismatch! Text: {len(text_embeddings)}, Image: {len(image_embeddings)}"

print(f"Text shape: {text_embeddings.shape}")
print(f"Image shape: {image_embeddings.shape}")

# Concatenate
print("\nConcatenating...")
hybrid_embeddings = np.concatenate([text_embeddings, image_embeddings], axis=1)

print(f"\n✅ Hybrid embeddings created!")
print(f"   Shape: {hybrid_embeddings.shape}")
print(f"   Expected: ({len(text_embeddings)}, {text_embeddings.shape[1] + image_embeddings.shape[1]})")
print(f"   Dimension: {hybrid_embeddings.shape[1]}d")

# Check for NaN
has_nan = np.isnan(hybrid_embeddings).any()
if has_nan:
    print("\n⚠️ WARNING: NaN values detected!")
    nan_count = np.isnan(hybrid_embeddings).sum()
    print(f"   NaN count: {nan_count}")
else:
    print("\n✅ No NaN values")

# Save
print("\nSaving hybrid embeddings...")
output_path = EMB_HYBRID_DIR / "hybrid_2304d.npy"
np.save(output_path, hybrid_embeddings)
print(f"✅ Saved: {output_path}")
print(f"   Size: {output_path.stat().st_size / 1024**2:.1f} MB")

🔗 CREATING HYBRID EMBEDDINGS...

Text shape: (44417, 1536)
Image shape: (44417, 768)

Concatenating...

✅ Hybrid embeddings created!
   Shape: (44417, 2304)
   Expected: (44417, 2304)
   Dimension: 2304d

✅ No NaN values

Saving hybrid embeddings...
✅ Saved: /content/drive/MyDrive/ai_fashion_assistant_v2/embeddings/hybrid/hybrid_2304d.npy
   Size: 390.4 MB


In [7]:
# ============================================================
# 7) NORMALIZE HYBRID EMBEDDINGS
# ============================================================

print("📐 NORMALIZING HYBRID EMBEDDINGS...\n")

from sklearn.preprocessing import normalize

# Normalize
hybrid_normalized = normalize(hybrid_embeddings, norm='l2')

print(f"✅ Normalized hybrid embeddings")
print(f"   Shape: {hybrid_normalized.shape}")
print(f"   Mean norm: {np.linalg.norm(hybrid_normalized, axis=1).mean():.4f}")

# Save
output_path = EMB_HYBRID_DIR / "hybrid_2304d_normalized.npy"
np.save(output_path, hybrid_normalized)
print(f"\n✅ Saved: {output_path}")
print(f"   Size: {output_path.stat().st_size / 1024**2:.1f} MB")

📐 NORMALIZING HYBRID EMBEDDINGS...

✅ Normalized hybrid embeddings
   Shape: (44417, 2304)
   Mean norm: 1.0000

✅ Saved: /content/drive/MyDrive/ai_fashion_assistant_v2/embeddings/hybrid/hybrid_2304d_normalized.npy
   Size: 390.4 MB


In [8]:
# ============================================================
# 8) BUILD FAISS INDEX (HNSW)
# ============================================================

print("🏗️ BUILDING FAISS INDEX...\n")
print("=" * 80)

# Use normalized embeddings for cosine similarity
embeddings_for_index = hybrid_normalized.astype('float32')

dimension = embeddings_for_index.shape[1]
n_vectors = embeddings_for_index.shape[0]

print(f"Index parameters:")
print(f"   Dimension: {dimension}")
print(f"   Vectors: {n_vectors:,}")
print(f"   Index type: HNSW (Hierarchical Navigable Small World)")

# HNSW parameters
M = 32  # Number of connections per layer
ef_construction = 200  # Build quality
ef_search = 100  # Search quality

print(f"\nHNSW parameters:")
print(f"   M: {M}")
print(f"   ef_construction: {ef_construction}")
print(f"   ef_search: {ef_search}")

# Create index
print("\nBuilding index...")
start_time = time.time()

index = faiss.IndexHNSWFlat(dimension, M)
index.hnsw.efConstruction = ef_construction
index.hnsw.efSearch = ef_search

# Add vectors
print("Adding vectors...")
index.add(embeddings_for_index)

elapsed = time.time() - start_time

print(f"\n✅ Index built successfully!")
print(f"   Time: {elapsed:.1f} seconds")
print(f"   Total vectors: {index.ntotal:,}")
print(f"   Is trained: {index.is_trained}")

🏗️ BUILDING FAISS INDEX...

Index parameters:
   Dimension: 2304
   Vectors: 44,417
   Index type: HNSW (Hierarchical Navigable Small World)

HNSW parameters:
   M: 32
   ef_construction: 200
   ef_search: 100

Building index...
Adding vectors...

✅ Index built successfully!
   Time: 10.8 seconds
   Total vectors: 44,417
   Is trained: True


In [9]:
# ============================================================
# 9) SAVE FAISS INDEX
# ============================================================

print("💾 SAVING FAISS INDEX...\n")

index_path = INDEX_DIR / "faiss_hybrid_hnsw.index"

# Save
faiss.write_index(index, str(index_path))

print(f"✅ Index saved: {index_path}")
print(f"   Size: {index_path.stat().st_size / 1024**2:.1f} MB")

💾 SAVING FAISS INDEX...

✅ Index saved: /content/drive/MyDrive/ai_fashion_assistant_v2/indexes/faiss_hybrid_hnsw.index
   Size: 401.9 MB


In [10]:
# ============================================================
# 10) TEST SEARCH FUNCTIONALITY
# ============================================================

print("🔍 TESTING SEARCH FUNCTIONALITY...\n")
print("=" * 80)

# Test with random query
test_idx = 42
test_query = hybrid_normalized[test_idx:test_idx+1].astype('float32')

print(f"Test query: Vector #{test_idx}")

# Search
k = 10
print(f"\nSearching for top-{k} results...")

start_time = time.time()
distances, indices = index.search(test_query, k)
search_time = (time.time() - start_time) * 1000  # ms

print(f"\n✅ Search completed!")
print(f"   Time: {search_time:.2f}ms")

print(f"\nTop-{k} results:")
print(f"{'Rank':<6} {'Index':<10} {'Distance':<12}")
print("-" * 30)
for i, (idx, dist) in enumerate(zip(indices[0], distances[0]), 1):
    similarity = 1 - dist  # Convert distance to similarity
    print(f"{i:<6} {idx:<10} {similarity:.6f}")

# Validate
print("\n✅ Validation:")
print(f"   First result is query itself: {indices[0][0] == test_idx}")
print(f"   Similarity ~1.0: {1 - distances[0][0] > 0.99}")

🔍 TESTING SEARCH FUNCTIONALITY...

Test query: Vector #42

Searching for top-10 results...

✅ Search completed!
   Time: 0.93ms

Top-10 results:
Rank   Index      Distance    
------------------------------
1      42         1.000000
2      28309      0.849985
3      42489      0.849597
4      7172       0.838907
5      3458       0.835715
6      1312       0.829875
7      39427      0.827966
8      24913      0.820948
9      24524      0.818811
10     40176      0.813978

✅ Validation:
   First result is query itself: True
   Similarity ~1.0: True


In [11]:
# ============================================================
# 11) BENCHMARK SEARCH PERFORMANCE
# ============================================================

print("⚡ BENCHMARKING SEARCH PERFORMANCE...\n")
print("=" * 80)

# Test with multiple queries
n_test_queries = 100
k = 10

print(f"Testing {n_test_queries} random queries...")
print(f"Retrieving top-{k} for each\n")

# Random queries
test_indices = np.random.randint(0, len(hybrid_normalized), n_test_queries)
test_queries = hybrid_normalized[test_indices].astype('float32')

# Benchmark
start_time = time.time()
distances, indices = index.search(test_queries, k)
elapsed = time.time() - start_time

avg_time_ms = (elapsed / n_test_queries) * 1000

print(f"✅ Benchmark results:")
print(f"   Total time: {elapsed:.2f}s")
print(f"   Average per query: {avg_time_ms:.2f}ms")
print(f"   Throughput: {n_test_queries / elapsed:.1f} queries/sec")

# QPS estimation
qps = 1000 / avg_time_ms
print(f"\n📊 Performance metrics:")
print(f"   QPS (Queries Per Second): {qps:.1f}")
print(f"   Latency (p50): ~{avg_time_ms:.2f}ms")

if avg_time_ms < 10:
    print(f"\n🚀 Excellent! Sub-10ms latency")
elif avg_time_ms < 50:
    print(f"\n✅ Good! Acceptable latency for production")
else:
    print(f"\n⚠️ Slow! Consider optimizing index parameters")

⚡ BENCHMARKING SEARCH PERFORMANCE...

Testing 100 random queries...
Retrieving top-10 for each

✅ Benchmark results:
   Total time: 0.02s
   Average per query: 0.17ms
   Throughput: 5955.8 queries/sec

📊 Performance metrics:
   QPS (Queries Per Second): 5955.8
   Latency (p50): ~0.17ms

🚀 Excellent! Sub-10ms latency


In [12]:
# ============================================================
# 12) GENERATE INDEX STATISTICS
# ============================================================

print("📊 GENERATING INDEX STATISTICS...\n")
print("=" * 80)

# Statistics
stats = {
    "index_type": "HNSW",
    "dimension": int(dimension),
    "total_vectors": int(index.ntotal),
    "parameters": {
        "M": M,
        "ef_construction": ef_construction,
        "ef_search": ef_search
    },
    "performance": {
        "avg_query_time_ms": float(avg_time_ms),
        "qps": float(qps)
    },
    "files": {
        "index": "faiss_hybrid_hnsw.index",
        "embeddings": "hybrid_2304d_normalized.npy"
    }
}

# Save stats
stats_path = INDEX_DIR / "index_stats.json"
with open(stats_path, 'w') as f:
    json.dump(stats, f, indent=2)

print(f"✅ Stats saved: {stats_path}")

# Print summary
print("\n📊 INDEX SUMMARY:")
print("=" * 80)
print(f"Index type: {stats['index_type']}")
print(f"Dimension: {stats['dimension']}d")
print(f"Total vectors: {stats['total_vectors']:,}")
print(f"\nPerformance:")
print(f"  Avg query time: {stats['performance']['avg_query_time_ms']:.2f}ms")
print(f"  QPS: {stats['performance']['qps']:.1f}")
print("=" * 80)

📊 GENERATING INDEX STATISTICS...

✅ Stats saved: /content/drive/MyDrive/ai_fashion_assistant_v2/indexes/index_stats.json

📊 INDEX SUMMARY:
Index type: HNSW
Dimension: 2304d
Total vectors: 44,417

Performance:
  Avg query time: 0.17ms
  QPS: 5955.8


In [13]:
# ============================================================
# 13) QUALITY GATES VALIDATION
# ============================================================

print("\n🎯 QUALITY GATES VALIDATION")
print("=" * 80)

gates_passed = True

# Gate 1: Hybrid dimension correct
expected_dim = text_embeddings.shape[1] + image_embeddings.shape[1]
if hybrid_embeddings.shape[1] == expected_dim:
    print(f"✅ Gate 1: Hybrid dimension correct ({expected_dim}d)")
else:
    print(f"❌ Gate 1: Dimension mismatch! Expected {expected_dim}, got {hybrid_embeddings.shape[1]}")
    gates_passed = False

# Gate 2: FAISS index built
if index.ntotal == len(hybrid_embeddings):
    print(f"✅ Gate 2: FAISS index built ({index.ntotal:,} vectors)")
else:
    print(f"❌ Gate 2: Index vector count mismatch!")
    gates_passed = False

# Gate 3: Index file saved
if index_path.exists():
    size_mb = index_path.stat().st_size / 1024**2
    print(f"✅ Gate 3: Index file saved ({size_mb:.1f} MB)")
else:
    print(f"❌ Gate 3: Index file not found!")
    gates_passed = False

# Gate 4: Search returns results
if indices[0][0] == test_idx and (1 - distances[0][0]) > 0.99:
    print(f"✅ Gate 4: Search returns correct results")
else:
    print(f"⚠️ Gate 4: Search results may be inaccurate")

# Gate 5: Performance acceptable
if avg_time_ms < 50:
    print(f"✅ Gate 5: Search performance acceptable ({avg_time_ms:.2f}ms)")
else:
    print(f"⚠️ Gate 5: Search slower than ideal ({avg_time_ms:.2f}ms)")

print("=" * 80)

if gates_passed:
    print("\n🎉 ALL QUALITY GATES PASSED!")
    print("✅ Hybrid space created successfully!")
    print("✅ FAISS index ready for retrieval!")
    print("\n🎊 PHASE 2 COMPLETE!")
    print("\n📍 Next: Phase 3 - Retrieval & Baseline Search")
else:
    print("\n⚠️ SOME QUALITY GATES FAILED!")
    print("   Please review and fix before proceeding.")


🎯 QUALITY GATES VALIDATION
✅ Gate 1: Hybrid dimension correct (2304d)
✅ Gate 2: FAISS index built (44,417 vectors)
✅ Gate 3: Index file saved (401.9 MB)
⚠️ Gate 4: Search results may be inaccurate
✅ Gate 5: Search performance acceptable (0.17ms)

🎉 ALL QUALITY GATES PASSED!
✅ Hybrid space created successfully!
✅ FAISS index ready for retrieval!

🎊 PHASE 2 COMPLETE!

📍 Next: Phase 3 - Retrieval & Baseline Search


---

## 📋 Summary

**Phase 2 Complete!** 🎊

**Files Created:**
- ✅ `embeddings/hybrid/hybrid_2304d.npy`
- ✅ `embeddings/hybrid/hybrid_2304d_normalized.npy`
- ✅ `indexes/faiss_hybrid_hnsw.index`
- ✅ `indexes/index_stats.json`

**Index Stats:**
- Type: HNSW
- Dimension: 2304d (1536 text + 768 image)
- Vectors: 44,417
- Performance: ~5-20ms per query
- Size: ~500 MB

**Next Phase:** Phase 3 - Retrieval
- Baseline search implementation
- Query processing
- Result ranking

---